In [2]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [6]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
		(104.0, 177.0, 123.0))

	faceNet.setInput(blob)
	detections = faceNet.forward()

	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		confidence = detections[0, 0, i, 2]

		if confidence > 0.5:
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			faces.append(face)
			locs.append((startX, startY, endX, endY))

	if len(faces) > 0:
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	return (locs, preds)

In [4]:
vs = cv2.VideoCapture('./data/02.mp4')
output_video = None

In [7]:
# load our serialized face detector model from disk
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

maskNet = load_model("face_mask_detection.h5")

print("[INFO] starting video...")

while True:
	(frame_exists, frame) = vs.read()
	if not frame_exists:
		break
	else:
		frame = imutils.resize(frame, width=400)

		(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

		for (box, pred) in zip(locs, preds):

			(startX, startY, endX, endY) = box
			value = pred

			label = "Mask" if value < 1  else "No Mask"
			color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

			cv2.putText(frame, label, (startX, startY - 10),
				cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
			cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	if output_video is None:
		fourcc1 = cv2.VideoWriter_fourcc(*"MJPG")
		output_video = cv2.VideoWriter("./output/result.avi", fourcc1, 25,(frame.shape[1], frame.shape[0]), True)
	else:
		print("writing to video")
		output_video.write(frame)

	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break
	
cv2.destroyAllWindows()

[INFO] starting video stream...
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
writing to video
